# 東京都福祉保健局のHPから新型コロナウイルスに関連した患者の発生について（過去1週間分）のPDFのURLをテキストに出力  
  
## 仕様  
- https://www.fukushihoken.metro.tokyo.lg.jp/hodo/saishin/hassei.html からpdfのパスを取得する
- htmlの解析はBeautifulsoupを使っている。  
- テキストは指定したフォルダへ保存する  

## 構成  
- Setting.text
  - 設定ファイル
    - 保存先の親フォルダを指定
- getURL.py
  - URLテキスト(json)を出力する
- getPDF.py
  - URLテキストからPDFを出力する
- getParseDataFromPDF.py
  - PDFファイルを読み込み、パースした情報を取得する

## URLテキスト(json)の形式

- name : PDFファイル名
- url : 取得元のURL
- isGetPDF : PDFファイルを取得している場合はTrue

### name



## import

In [1]:
import os
import sys
import json
from WebScrapingTool import Base_UserFunction as uf
import comFunction


## 基本パーツ

In [2]:
class base:
    settingDict = dict()
    prevPressURL = list()
    saveData = list()

    tagSaveDir = '[0]'
    tagDebug = '[a]'
    tagSaveFolder = '[b]'
    tagSaveFileName = '[c]'
    tagBaseURL = '[d]'
    tagTopURL = '[e]'
    tagPreviousPDFLinkURL = '[f]'
    tagTypeLast = '[g]'
    tagTypePrevious = '[h]'
    tagDataFormat = '[i]'
    tagPrevPressURL = '[m]'

    # コンストラクタ
    def __init__(self):
        pass
    
    def setSettingData(self, com):
        # 設定ファイルから必要な情報を取得する
        
        # 先頭は必ず SaveDir にする
        self.settingDict = com.getSettingData([
            self.tagSaveDir,
            self.tagDebug,
            self.tagSaveFolder,
            self.tagSaveFileName,
            self.tagBaseURL,
            self.tagTopURL,
            self.tagPreviousPDFLinkURL,
            self.tagTypeLast,
            self.tagTypePrevious,
            self.tagDataFormat,
            self.tagPrevPressURL
            ])        
        print(self.settingDict)

        if len(self.settingDict) <= 0:
            com.errMsg(__name__, 'SettingData is none...')
            return

        if self.settingDict[self.tagDebug] == 'true':
            com.setDebug(True)
        else:
            com.setDebug(False)
        
    def checkInitialize(self):
        if len(self.settingDict[self.tagSaveFolder]) <= 0:
            print('[!!!ERROR!!!] Save FolderName is Empty!')
            return False    
        if len(self.settingDict[self.tagSaveFileName]) <= 0:
            print('[!!!ERROR!!!] Save FileName is Empty!')
            return False    
        return True        
        
    def createSaveFolderAndFile(self):
        if not os.path.exists(self.settingDict[self.tagSaveFolder]):
            os.mkdir(self.settingDict[self.tagSaveFolder])
        oFile = self.settingDict[self.tagSaveFolder] + '/' + self.settingDict[self.tagSaveFileName]
        if not os.path.exists(oFile):
            f = open(oFile,'w')
            f.close()


    def getSettingData(self):
        return \
        self.settingDict[self.tagBaseURL], \
        self.settingDict[self.tagTopURL], \
        self.settingDict[self.tagPreviousPDFLinkURL], \
        self.settingDict[self.tagTypeLast], \
        self.settingDict[self.tagTypePrevious], \
        self.settingDict[self.tagDataFormat]

    def getPrevURL(self):
        return self.prevPressURL

    def setSaveData(self, _data):
        self.saveData.append(_data)
        
    def saveGetData(self):
        print('[Save Data]')
        if len(self.saveData) <= 0:
            print('No Data...')
            return
        
        # リスト作成
        oFile = self.settingDict[self.tagSaveDir] + '/' \
        + self.settingDict[self.tagSaveFolder] + '/' \
        + self.settingDict[self.tagSaveFileName]

        # 重複データは再度追加しない
        _alreadyGetURL = list()
        with open(oFile, mode='r') as fs:
            for line in fs:
                if len(line) <= 0:
                    print("Size Zero")  
                    continue
                if not ( set(('{', '}')) <= set(line)):
                    print("Not Json Format :" + line)  
                    continue

                j = json.loads(line)
                _alreadyGetURL.append(j['url'])
                
        with open(oFile, mode='a') as fs:
            for line in self.saveData:
                j = json.loads(line)
                if not j['url'] in _alreadyGetURL:
                    uf.fileWrite(fs, line)
                else:
                    pass
#                    print('already get url : ' + line)
        # 重複データ削除    
        uf.fileDataSlim(oFile)    
    

## スクレイピング本体

In [3]:
class work:
    baseURL = ''
    topURL = ''
    previousPDFLinkURL = ''
    typeLast = ''
    typePrevious = ''    
    PDFPath = ''
    dataFormat = ''
    #コンストラクタ
    def __init__(self, b):
        self.PDFPath = '<p class="pagelinkout">'
        self.baseURL, self.topURL, self.previousPDFLinkURL, self.typeLast, self.typePrevious, self.dataFormat = b.getSettingData()

    def setSaveData(self, url, isLast):
        name = url.replace("/tosei", "")
        name = name.replace("/hodohappyo", "")
        name = name.replace("/press", "")
        name = name.replace("/documents", "_")
        name = name.replace("/", "")
        fullURL = ''
        type = ''
        if isLast:
            type = self.typeLast
            fullURL = self.baseURL + '/hodo/saishin/' + url
        else:
            type = self.typePrevious
            fullURL = self.previousPDFLinkURL + url 

        _saveData = self.dataFormat + '\n'
        _saveData = _saveData.replace("@@type", type)
        _saveData = _saveData.replace("@@name", name)
        _saveData = _saveData.replace("@@url", fullURL)
        return _saveData
        
    def getURLandSetData(self, b):
        _html = uf.getHTML(self.baseURL + self.topURL)
        bs = uf.getBS4(_html)
        #d.printLog(bs.body)

        _linkLastPath = ""
        _linkPrevPath = b.getPrevURL()
        _bsLinkPathList = bs.findAll("p", {"class": "pagelinkout"})
        for _linkList in _bsLinkPathList:
            # last Data
            _bsLinkLastPath = _linkList.find("a", {"class": "innerLink"})
            if _bsLinkLastPath is not None:
                _linkLastPath = self.baseURL + _bsLinkLastPath["href"]

            # previous Data
            _bsLinkPath = _linkList.find("a", {"class": "externalLink"})
            if _bsLinkPath is not None:
                _linkPrevPath.append(_bsLinkPath["href"])
            
        if len(_linkLastPath) <= 0:
            print('[!!!ERROR!!!] Search LinkPath(Last) is Empty!')
            return False
        if len(_linkPrevPath) <= 0:
            print('[!!!ERROR!!!] Search LinkPath(Previous) is Empty!')
            return False

        # Last PDF Link
        print("[get Last PDF Link]")
        print(_linkLastPath)
        _html = uf.getHTML(_linkLastPath)
        bs = uf.getBS4(_html)
        _bsPDFPathList = bs.findAll("a", {"class":"resourceLink newWindow"})
        isLast = True
        for _url in _bsPDFPathList:
            b.setSaveData(self.setSaveData(_url["href"], isLast))
        
        # previous PDF Link
        print("[get Previous PDF Link]")
        for _link in _linkPrevPath:
            print(_link)
            _html = uf.getHTML(_link)
            bs = uf.getBS4(_html)
            _bsPDFPathList = bs.findAll("a", {"class":"icon_pdf"})
            isLast = False
            for _url in _bsPDFPathList:
                b.setSaveData(self.setSaveData(_url["href"], isLast))
        
        return True

## 最初に呼ばれる

In [4]:
def main():  
    print("\n[Start]" + uf.getNowTime() + '\n')            

    com = comFunction.common()

    b = base()
    b.setSettingData(com)
    
    if not b.checkInitialize():
        return
    
    w = work(b)
    
    # 保存フォルダ,ファイル作成
    b.createSaveFolderAndFile()
    
    w.getURLandSetData(b)
    b.saveGetData()
    
    
    print("\n[ End ]" + uf.getNowTime() + '\n')
    


## 処理開始

In [5]:
if __name__ == '__main__':
    main()


[Start]20211214225658

INFO [comFunction] SaveDir : d:\#WorkSpace\ReadCovid-19/
{'[0]': 'd:\\#WorkSpace\\ReadCovid-19/', '[a]': 'True', '[b]': 'list', '[c]': 'dataList.json', '[d]': 'https://www.fukushihoken.metro.tokyo.lg.jp', '[e]': '/hodo/saishin/hassei.html', '[f]': 'https://www.metro.tokyo.lg.jp', '[g]': 'Last', '[h]': 'Previous', '[i]': '{"type" : "@@type", "name" : "@@name", "url" : "@@url", "isGetPDF" : "False", "isParse" : "False" }', '[m]': 'prevPressURL.txt'}
[get Last PDF Link]
https://www.fukushihoken.metro.tokyo.lg.jp/hodo/saishin/corona2725.html
[get Previous PDF Link]
https://www.metro.tokyo.lg.jp/tosei/hodohappyo/press/2021/12/14/21.html
https://www.metro.tokyo.lg.jp/tosei/hodohappyo/press/2021/12/13/20.html
https://www.metro.tokyo.lg.jp/tosei/hodohappyo/press/2021/12/12/01.html
https://www.metro.tokyo.lg.jp/tosei/hodohappyo/press/2021/12/11/01.html
https://www.metro.tokyo.lg.jp/tosei/hodohappyo/press/2021/12/10/21.html
https://www.metro.tokyo.lg.jp/tosei/hodohappyo/p